# Azure Cost Model
In this notebook, we go through the cost model in "Assessing requirements to scale to practical quantum advantage" by [Beverland et al](https://arxiv.org/abs/2211.07629) to reproduce the estimated costs in Appendix F.

The paper describes the formulas used for estimating cost in the various appendices. The final estimation procedure is put together in Appendix E. Throughout, we reproduce the costs from the paper; in particular Appendix F.

The Azure cost model has 5 inputs:
1. The physical assumptions about the hardware (e.g. error rate, the latency of Clifford and measurement operations, etc).
1. A summary of the circuit/algorithm to execute (e.g. number of T gates, etc).
1. The magic state factory (e.g. number of qubits consumed by it, its error rate, etc).
1. Cost model of Approximating rotations using T operations up to error $\epsilon$.
1. The quantum error correction scheme.

We take a look at each of these and then reproduce the results of Appendix F.

## The Inputs
### Physical Parameters
These are assumptions about the quantum hardware and are:
- `t_gate_ns`: Latency time of Clifford gates. 
- `t_meas_ns`: Latency time of Measurement operations.
- `physical_error_rate`: Physical error rate ($p$).

In Qualtran these are represented by the `PhysicalParameters` dataclass.

In [ ]:
from qualtran.surface_code.physical_parameters import BEVERLAND_PARAMS

BEVERLAND_PARAMS

### Algorithm Summary
This is a summary of the circuit or algorithm to execute. This summary is several simple counts:
- `algorithm_qubits` is the number of algorithm qubits.
- `measurements` is the number of Clifford measurements ($M_R$).
- `t_gates` is the number of T gates ($M_T$).
- `toffoli_gates` is the number of Toffoli gates ($M_{Tof}$).
- `rotation_gates` is the number of rotations ($M_R$).
- `rotation_circuit_depth` is the depth of rotation circuit  ($D_R$).

Note: The symbol in parentheses corresponds to the notation in the paper

In Qualtran the algorithm specs are represented by the data class `qualtran.surface_code.AlgorithmSummary`.

In [ ]:
from qualtran.surface_code.algorithm_summary import AlgorithmSummary

# A circuit that applies a T then measures the qubit.
circuit_summary = AlgorithmSummary(algorithm_qubits=1, t_gates=1, measurements=1)
circuit_summary

### Magic State Factory

The magic state factory in our case is a T-state factory. The paper describes 15-to-1 factories in Appendix C, but only the overall summary in Table VII in terms of physical qubit count and generation time is used in estimation in Appendix F.

### Rotation Approximation Model

This is a model that approximates the number of T gates needed to implement a rotation up to error $\epsilon$. In the paper they use the approximation model from [Kliuchnikov et al](https://arxiv.org/abs/2203.10064). The formula for the number of T gates used to approximate an angle up to error $\epsilon$ is given by:
$$
a \log_2{\frac{1}{\epsilon}} + b
$$

Where $a$ and $b$ are constants that depend on the gate set and the approximation protocol. Table 1 of [Kliuchnikov et al](https://arxiv.org/abs/2203.10064) gives estimates for those constants for different combinations of gate sets and protocols.

In the paper, they use $a=0.53, b=5.3$

In [ ]:
from qualtran.surface_code.rotation_cost_model import BeverlandEtAlRotationCost

BeverlandEtAlRotationCost

### Quantum Error Correction Scheme
The quantum error correction scheme determines three things:
1. The logical error rate given a code distance and physical error rate: $P(d)$.
1. The number of physical qubits needed per logical qubit: $n(d)$.
1. The number of logical time steps: $\tau(d)$.

Table V of the paper lists how these are related to the QEC scheme.

In the paper, they use gate-based QEC which has $P(d), n(d), \tau(d)$ as:
$$
P(d) = 0.03 \left ( \frac{p}{0.01} \right) ^ \frac{d+1}{2}\\
n(d) = 2 d^2\\
\tau(d) = \textrm{\{single stabilizer time\}} \cdot d\\
$$

The error detection circuit time depends on several factors physical factors including the time to apply a Clifford, measurement and reset operations as well as classical processing.

In Table V they don't take into account the classical processing part and assume that a reset takes the same time as a measurement leading to the formula:
$$
\textrm{\{single stabilizer time\}} = 4t_\textrm{gate} + 2t_\textrm{meas}
$$

Other authors (e.g. [Fowler, Gidney](https://arxiv.org/abs/1808.06709)) assume that the entire process takes a specific time (e.g. $\approx 1\mu s$).

In Qualtran, we use "single stabilizer time" rather than fall to the low-level hardware parameters. We also default to the Fowler, Gidney parameters (i.e. `FowlerSuperconductingQubits`) when none are given.

In [ ]:
from qualtran.surface_code.quantum_error_correction_scheme_summary import (
    BeverlandSuperconductingQubits,
)

qec = BeverlandSuperconductingQubits
qec

## Resource Estimation
Now we move to reproduce the results in Appendix F.

### Quantum Dynamics
The algorithm specs of this circuit are given as:
- number of algorithm qubits: $100$
- number of rotation gates: $30,100$
- number of measurements: $1.4 \times 10^6$
- number of T gates: $0$
- number of Toffoli gates: $0$
- depth of rotation circuit: $501$

with an error budget $\epsilon$ of $0.001$

In [ ]:
quantum_dynamics_specs = AlgorithmSummary(
    algorithm_qubits=100,
    rotation_gates=30100,
    # Note in the paper the number of measurements
    # has an extra zero which we assume to be a typo.
    measurements=1.4e5,
    rotation_circuit_depth=501,
)
quantum_dynamics_specs

In [ ]:
from qualtran.surface_code import azure_cost_model

# First calculate the number of logical qubits (Eq. D1)
logical_qubits = azure_cost_model.logical_qubits(quantum_dynamics_specs)
print('Q =', logical_qubits)

In [ ]:
# Calculate the minimum number of logical time steps (Eq D3)
error_budget = 0.001
c_min = azure_cost_model.minimum_time_steps(
    error_budget=error_budget, alg=quantum_dynamics_specs, rotation_model=BeverlandEtAlRotationCost
)
print('C_min = %e' % c_min)

In [ ]:
# And the number of needed T operations (Eq D4)
t_operations = azure_cost_model.t_states(
    error_budget=error_budget, alg=quantum_dynamics_specs, rotation_model=BeverlandEtAlRotationCost
)
print('M = %e' % t_operations)

Comparing our esimates of $Q = 230, C_{min} = 1.8 \times 10^5, M = 6 \times 10^5$ to the paper estimates of 
$Q = 230, C_{min} = 1.5 \times 10^5, M = 2.4 \times 10^6$. We find a match for $Q$ and $C_{min}$ however we are off by 4x for the number of T gates $M$.

D4 gives the formula for $M$ as $$M = M_T + 4 M_{Tof} + M_R \lceil A \log_2{M_R/\epsilon_{syn}} + B\rceil$$

Since $M_T = M_{Tof} = 0$, the only contribution to $M$ comes from rotations where $M_R = 30100, \epsilon_{syn} = \frac{10^{-3}}{3} , A = 0.53, B = 5.3$ which give our value of $6.02e5$.

Next, we estimate the code distance $d$. $d$ satisfies 
$$
P(d) = a \left ( \frac{p}{p^*} \right )^\frac{d+1}{2} 
$$
subject ot the constraint on the logical error rate of $Q \cdot C \cdot P(d) = \frac{\epsilon}{3}$. Where $p$ is the physical error rate. $a$ and $p^*$ are constants determined by the QEC scheme.

In [ ]:
d = azure_cost_model.code_distance(
    error_budget=error_budget,
    time_steps=c_min,
    alg=quantum_dynamics_specs,
    qec=BeverlandSuperconductingQubits,
    physical_error_rate=BEVERLAND_PARAMS.physical_error_rate,
)
print(f'{d=}')

Matching the paper's code distance of $d = 9$. This leads to a total run time (Eq. E3) of 0.65s which is close to the time in the paper of 0.55s

In [ ]:
t_s = qec.error_detection_circuit_time_us(d) * 1e-6 * c_min
f'algorithm run time of {t_s:g} seconds'.format()

Next, we examine the magic state factories. In the paper, for the quantum dynamics example, we are given $199$ factories each producing one T state every $46.8 \mu s$ at an error rate of $5.6e-11$ while consuming $3,240$ qubits. 

In [ ]:
num_factories = 199
factory_qubits = 3240

In [ ]:
# Leading to a total number of physical qubits (from E6)
distillation_qubits = num_factories * factory_qubits
q = distillation_qubits + logical_qubits * qec.physical_qubits(d)
print('total number of physical qubits:', q)
print('percentage of distillation qubits: {}%'.format(round(distillation_qubits / q * 100, 1)))

Our estimate of 0.68M physical qubits with 94.5% of them being consumed by the T states factories match the numbers in the paper.

### Quantum Chemistry
The algorithm specs of this circuit are given as:
- number of algorithm qubits: 1318
- number of rotation gates: $2.06 \times 10^8$
- number of measurements: $1.37 \times 10^9$
- number of T gates: $5.53 \times 10^7$
- number of Toffoli gates: $1.35 \times 10^{11}$
- depth of rotation circuit: $2.05 \times 10^8$

with an error budget of 0.01

In [ ]:
quantum_chemistry_specs = AlgorithmSummary(
    algorithm_qubits=1318,
    rotation_gates=2.06e8,
    measurements=1.37e9,
    toffoli_gates=1.35e11,
    t_gates=5.53e7,
    rotation_circuit_depth=2.05e8,
)
quantum_chemistry_specs

In [ ]:
logical_qubits = azure_cost_model.logical_qubits(quantum_chemistry_specs)
print('Q =', logical_qubits)

In [ ]:
# Calculate the minimum number of logical time steps (Eq D3)
error_budget = 0.01
c_min = azure_cost_model.minimum_time_steps(
    error_budget=error_budget, alg=quantum_chemistry_specs, rotation_model=BeverlandEtAlRotationCost
)
print('C_{min} = %g' % c_min)

In [ ]:
# And the number of needed T operations (Eq D4)
t_operations = azure_cost_model.t_states(
    error_budget=error_budget, alg=quantum_chemistry_specs, rotation_model=BeverlandEtAlRotationCost
)
print('M = %g' % t_operations)

Note that our estimates match nicely to the paper estimates of 
$$Q = 2740\\
C_{min} = 4.10 \times 10^{11}\\
M = 5.44 \times 10^{11}$$

Next, we estimate the code distance.

In [ ]:
d = azure_cost_model.code_distance(
    error_budget=error_budget,
    time_steps=c_min,
    alg=quantum_chemistry_specs,
    qec=qec,
    physical_error_rate=BEVERLAND_PARAMS.physical_error_rate,
)
print(f'{d=}')

Again we match the code distance from the paper of $d = 17$. This leads to a total run time (Eq. E3) of

In [ ]:
total_seconds = qec.error_detection_circuit_time_us(d) * 1e-6 * c_min
total_days = total_seconds / 3600 / 24
'algorithm run time of %g days' % (total_days)

We get a run time estimate of 32.4 days. In the paper, it says the run time is 1 month and 1 day.

Next we examine the magic state factories. In the paper, for the quantum chemistry example, we are given $17$ factories each producing one T state every $83.2\mu s$ at an error rate of $2.13e-15$ while consuming $16,000$ qubits. 

In [ ]:
num_factories = 17
factory_qubits = 16000

In [ ]:
# Leading to a total number of physical qubits (from E6)
distillation_qubits = num_factories * factory_qubits
q = distillation_qubits + logical_qubits * qec.physical_qubits(d)
print('total number of physical qubits: %g M' % round(q * 1e-6, 2))
print('percentage of distillation qubits: {}%'.format(round(distillation_qubits / q * 100, 1)))

Our estimate of 1.86M physical qubits matches the paper's estimate.

### Shor Factoring
The algorithm specs of this circuit are given as:
- number of algorithm qubits: $12,581$
- number of rotation gates: $12$
- number of measurements: $1.08 \times 10^9$
- number of T gates: 12
- number of Toffoli gates: $3.73 \times 10^{10}$
- depth of rotation circuit: $12$

with an error budget of $1/3$

In [ ]:
shor_specs = AlgorithmSummary(
    algorithm_qubits=12581,
    rotation_gates=12,
    measurements=1.08e9,
    rotation_circuit_depth=12,
    # Note in the paper the number of Toffoli operations is 3.73e10.
    # However we assume that the exponent has a typo and that the number is 3.73e9.
    toffoli_gates=3.73e9,
    t_gates=12,
)

In [ ]:
logical_qubits = azure_cost_model.logical_qubits(shor_specs)
print('Q =', logical_qubits)

In [ ]:
# Calculate the minimum number of logical time steps (Eq D3)
error_budget = 1 / 3
c_min = azure_cost_model.minimum_time_steps(
    error_budget=error_budget, alg=shor_specs, rotation_model=BeverlandEtAlRotationCost
)
print('C_min = %e' % c_min)

In [ ]:
# And the number of needed T operations (Eq D4)
t_operations = azure_cost_model.t_states(
    error_budget=error_budget, alg=shor_specs, rotation_model=BeverlandEtAlRotationCost
)
print('M = %e' % t_operations)

Our estimates of $Q = 25481, C_{min} = 1.23 \cdot 10^{10}, M = 1.49 \cdot 10^{10}$ match the estimates of the paper.

In [ ]:
d = azure_cost_model.code_distance(
    error_budget=error_budget,
    time_steps=c_min,
    alg=shor_specs,
    qec=qec,
    physical_error_rate=BEVERLAND_PARAMS.physical_error_rate,
)
print(f'{d=}')

Matching the code distance of $d = 13$ in the paper.

In [ ]:
total_seconds = qec.error_detection_circuit_time_us(d) * 1e-6 * c_min
total_hours = total_seconds / 3600
h = int(total_hours)
m = (total_hours - h) * 60
'algorithm run time of %d hours %d' % (h, m)

Our estimate runtime of 17h43m matches with the estimate of the paper.

Next we examine the magic state factories. In the paper, for the quantum chemistry example, we are given $18$ factories each producing one T state every $72.8 \mu s$ at an error rate of $5.51e-13$ while consuming $5,760$ qubits. 

In [ ]:
num_factories = 18
factory_qubits = 5760

In [ ]:
# Leading to a total number of physical qubits (from E6)
distillation_qubits = num_factories * factory_qubits
q = distillation_qubits + logical_qubits * qec.physical_qubits(d)
print('total number of physical qubits: %g M' % round(q * 1e-6, 2))
print('percentage of distillation qubits: {}%'.format(round(distillation_qubits / q * 100, 1)))

Our estimate of 8.72M physical qubits matches the paper's estimate.